In [ ]:
#import necessary libraries
import numpy as np                       
from sklearn.model_selection import train_test_split
import pickle
from sklearn.utils import shuffle
import cv2
from keras.models import Sequential
from keras.layers import Activation, Dropout, UpSampling2D
from keras.layers import Conv2DTranspose, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from keras.models import load_model

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)876544/45929032 bytes (1.9%)2383872/45929032 bytes (5.2%)4243456/45929032 bytes (9.2%)6701056/45929032 bytes (14.6%)9314304/45929032 bytes (20.3%)11976704/45929032 bytes (26.1%)14311424/45929032 bytes (31.2%)16883712/45929032 bytes (36.8%)19488768/45929032 bytes (42.4%)21987328/45929032 bytes (47.9%)24625152/45929032 bytes (53.6%)27344896/45929032 bytes (59.5%)29958144/

In [ ]:
train_images = pickle.load(open("full_CNN_train.p", "rb" ))

In [ ]:
train_images= train_images[0:5]  #slicing to take less images

In [ ]:
labels = pickle.load(open("full_CNN_labels.p", "rb" ))

In [ ]:
labels=labels[0:5]    #slicing to take less labels

In [ ]:
def create_model(input_shape, pool_size):
  model = Sequential()
  model.add(BatchNormalization(input_shape=input_shape))
  model.add(Conv2D(8, (3, 3), padding='valid', strides=(1,1), activation = 'relu'))    #conv layer1 
  model.add(Conv2D(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu'))   #conv layer 2
  model.add(MaxPooling2D(pool_size=pool_size))                                         #pooling layer 1
  model.add(Conv2D(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu'))   #conv layer 3
  model.add(Dropout(0.2))
  model.add(Conv2D(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu'))   #conv layer 4
  model.add(Dropout(0.2))
  model.add(Conv2D(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu'))   #conv layer 5
  model.add(Dropout(0.2))
  model.add(MaxPooling2D(pool_size=pool_size))                                         #pooling layer 2
  model.add(Conv2D(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu'))   #conv layer 6
  model.add(Dropout(0.2))
  model.add(Conv2D(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu'))   #conv layer 7
  model.add(Dropout(0.2))
  model.add(MaxPooling2D(pool_size=pool_size))                                         #pooling layer 3
  model.add(UpSampling2D(size=pool_size))                                              #upsample layer 1
  model.add(Conv2DTranspose(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu')) #deconv layer 1
  model.add(Dropout(0.2))
  model.add(Conv2DTranspose(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv2'))  #deconv layer 2
  model.add(Dropout(0.2))
  model.add(UpSampling2D(size=pool_size))                                               #upsample layer 2
  model.add(Conv2DTranspose(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu')) #deconv layer 3
  model.add(Dropout(0.2))
  model.add(Conv2DTranspose(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu'))  #deconv layer 4
  model.add(Dropout(0.2))
  model.add(Conv2DTranspose(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu'))  #deconv layer 5
  model.add(Dropout(0.2))
  model.add(UpSampling2D(size=pool_size))                                                      #upsample layer 3
  model.add(Conv2DTranspose(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu'))  #deconv layer 6
  model.add(Conv2DTranspose(1, (3, 3), padding='valid', strides=(1,1), activation = 'relu'))   #final layer including one channel only
  return model

In [ ]:
def main():
  train_images = pickle.load(open("full_CNN_train.p", "rb" ))  #loading training images
  labels = pickle.load(open("full_CNN_labels.p", "rb" ))       #loading image labels
  train_images = np.array(train_images)   #making arrays for neural network
  labels = np.array(labels)
  labels = labels / 255     #normalizing labels
  train_images, labels = shuffle(train_images, labels)  #shuffling images along with its labels
  X_train, X_val, y_train, y_val = train_test_split(train_images, labels, test_size=0.1)  #splitting in training and validation sets
  batch_size = 32   #optimizing
  epochs = 10
  pool_size = (2, 2)
  input_shape = X_train.shape[1:]
  model = create_model(input_shape, pool_size)    #creating the model
  datagen = ImageDataGenerator(channel_shift_range=0.2)   #generator to help model use less data
  datagen.fit(X_train)
  model.compile(optimizer='Adam', loss='mean_squared_error')   #compiling and training the model
  model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), steps_per_epoch=len(X_train)/batch_size,
  epochs=epochs, verbose=1, validation_data=(X_val, y_val))
  model.trainable = False       #freezing layer as training is done
  model.compile(optimizer='Adam', loss='mean_squared_error')
  model.save('full_CNN_model.h5')  #saving the model
  model.summary()       #summarzing the model
if __name__ == '__main__':
    main()

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
358/358 [==============================] - 752s 2s/step - loss: 0.0517 - val_loss: 0.0098
Epoch 2/10
358/358 [==============================] - 775s 2s/step - loss: 0.0101 - val_loss: 0.0082
Epoch 3/10
358/358 [==============================] - 771s 2s/step - loss: 0.0086 - val_loss: 0.0075
Epoch 4/10
358/358 [==============================] - 768s 2s/step - loss: 0.0081 - val_loss: 0.0073
Epoch 5/10
358/358 [==============================] - 768s 2s/step - loss: 0.0074 - val_loss: 0.0055
Epoch 6/10
358/358 [==============================] - 764s 2s/step - loss: 0.0061 - val_loss: 0.0052
Epoch 7/10
358/358 [==============================] - 764s 2s/step - loss: 0.0057 - val_loss: 0.0051
Epoch 8/10
358/358 [==============================] - 769s 2s/step - loss: 0.0056 - val_loss: 0.0050
Epoch 9/10
358/358 [==============================] - 767s 2s/step - loss: 0.0051 - val_loss: 0.0047
Epoch 10/10
358/358 [==============================] - 764s 2s/step - loss: 0.0052 - val_lo

In [ ]:
class Lanes():
    def __init__(self):
        self.recent_fit = []
        self.avg_fit = []

In [ ]:
from skimage.transform import resize
def road_lines(image):
    """ Takes in a road image, re-sizes for the model,
    predicts the lane to be drawn from the model in G color,
    recreates an RGB image of a lane and merges with the
    original road image.
    """

    # feeding image in model
    small_img = resize(image, (80, 160, 3))
    small_img = np.array(small_img)
    small_img = small_img[None,:,:,:]

    # Making prediction with neural network 
    prediction = model.predict(small_img)[0] * 255  #un-normalize value by multiplying by 255

    # Adding lane prediction 
    lanes.recent_fit.append(prediction)
    # Only using last three for average
    if len(lanes.recent_fit) > 3:
        lanes.recent_fit = lanes.recent_fit[1:]

    # Calculate average detection
    lanes.avg_fit = np.mean(np.array([i for i in lanes.recent_fit]), axis = 0)

    # Generate fake R & B color dimensions, stacking with G
    blanks = np.zeros_like(lanes.avg_fit).astype(np.uint8)
    lane_drawn = np.dstack((blanks, lanes.avg_fit, blanks))

    # Re-size to match the original image
    lane_image = resize(lane_drawn, (720, 1280, 3))

    # Merge the lane drawing onto the original image
    result = cv2.addWeighted(image, 1, lane_image, 1, 0)

    return result


In [ ]:
import tensorflow as tf
if __name__ == '__main__':
    
    model = tf.keras.models.load_model('full_CNN_model.h5')
    # Create lanes object
    lanes = Lanes()

    # saving output video
    vid_output = 'output_vid.mp4'
    # input video
    clip1 = VideoFileClip("input_video.mp4")
    # Create the clip
    vid_clip = clip1.fl_image(road_lines)
    vid_clip.write_videofile(vid_output, audio=False)

error: ignored